In [53]:
import torch
import supervision as sv
import cv2
from ultralytics import YOLO
import numpy as np
print(torch.__version__)        # should show +cu128
print(torch.version.cuda)       # should match runtime CUDA version
print(torch.cuda.is_available())  # True if GPU detected

2.9.1+cu128
12.8
True


In [54]:
VIDEO_PATH = "../VIDEOSYOLO/video3.mp4"

model = YOLO("yolo11n.pt")

frames_generator = sv.get_video_frames_generator(
    source_path=VIDEO_PATH,
)

POLYGON= np.array([[230.55225148683093, 2.463891248937978],
 [322.9906542056075, 108.49617672047579],
 [383.89124893797793, 229.20985556499576],
 [365.94732370433303, 349.37977909940525],
 [438.266779949023, 246.06627017841973],
 [544.2990654205607, 165.0467289719626],
 [640, 129.1588785046729],
 [636.1937128292269, 186.2531860662702],
 [602.480883602379, 212.3534409515718],
 [541.5802888700085, 360],
 [258.2837723024639, 358.6236193712829],
 [275.6839422259983, 135.6839422259983],
 [232.18351741716228, 49.226847918436704],
 [247.95242141036533, 113.93372982158029],
 [218.58963466440102, 359.7111299915038],
 [0, 356.4485981308411],
 [0, 294.4604927782498],
 [156.05777400169924, 96.53355989804588],
 [172.9141886151232, 0.2888700084961767]], dtype=np.int32)


polygon_zone = sv.PolygonZone(POLYGON, triggering_anchors=(sv.Position.CENTER,))
box_annotator = sv.BoxAnnotator()

for idx, frame in enumerate(frames_generator):
    result = model(frame, device="cuda", verbose=False)[0]
    detections = sv.Detections.from_ultralytics(result)
    #print(detections)
    detections = detections[polygon_zone.trigger(detections)]

    # Dibujar cajas
    annotated_frame = frame.copy()
    annotated_frame = sv.draw_polygon(scene=annotated_frame, polygon=POLYGON, color=sv.Color.YELLOW, thickness=2)
    annotated_frame = box_annotator.annotate(
        scene=annotated_frame,
        detections=detections
    )

    # Mostrar frame anotado
    cv2.imshow("Annotated Frame", annotated_frame)

    # salir con 'q'
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
